## read_general

### **概要**
この関数は関数`read_jw`をベースとして改良し、OpenfermionによるJordan-Wigner変換結果のみならず、Bravyi-Kitaev変換結果に対しても係数及びPauli演算子の情報を読み取るものである\
汎用性の観点から今後はこちらの関数の使用を推奨する

### **引数一覧**
|argument name|type|role|
|---|---|---|
|jw_inf|list(elements:str)|OpenfermionによるJordan-Wigner変換もしくはBravyi-Kitaev変換結果|

### **戻り値**
num_list（list）:\
引数のうち係数に関する情報を格納したlist\
各要素はcomplex型をとる\
ope_list（list）:\
引数のうちPauli行列積に関する情報を格納したlist\
各要素はstr型をとる

### **Python code**
```python
def read_general(jw_inf):
    """
    This function reads out the coefficients and Pauli matrix product information from the results of the Jordan–Wigner and Bravyi–Kitaev transformations performed using OpenFermion.

    Parameters:
        jw_inf: the results of the Jordan–Wigner and Bravyi–Kitaev transformations performed using OpenFermion

    Returns:
        (list, list): The coefficients information and Pauli matrix product information.
    """
    # Local Values
    num_list = [] # A list to store the results of the reading process for coefficients
    ope_list = [] # A list to store the results of the reading process for Pauli matrix product
    sub_num_list = "" # A temporary list to store the results of the reading process for coefficients
    sub_ope_list = "" # A temporary list to store the results of the reading process for Pauli matrix product
    convert_list = [] # A temporary list to store the results of the reading process for coefficients before transform the type of coeffcient information from str into complex
    ope_con_list = [] # A temporary list to store the results of the reading process for Pauli matrix product before deleting space between Pauli matrix product
    read_counter = 1 # A switch to initiate reading
    preserve_switch = 0 # A switch to store the contents of sum_num_list into convert_list
    # Check each contents (excluding the first) to determine whether to interpret it as a coefficient, as part of a Pauli matrix product, or to ignore it.
    for i in range(len(jw_inf)-1):
        # Determine whether to interpret each item as a coefficient, a Pauli matrix product, or to ignore it.
        # 1: coefficient, 2: Pauli matrix product, 3, 4: ignore
        if jw_inf[i+1] == "[":
            read_counter = 2
        elif jw_inf[i+1] == "]":
            read_counter = 3
        # Read the coefficient as type str (ex: (0.2450065090650131+0j))
        if read_counter == 1:
            sub_num_list += jw_inf[i]
            preserve_switch = 1
        # Read the Pauli matrix product as type str (ex: 'X0 X1 Y2 Y3')
        if read_counter == 2:
            # Store the result of reading for the coefficient
            if preserve_switch == 1:
                convert_list.append(sub_num_list)
                sub_num_list = ""
                preserve_switch = 0
            sub_ope_list += jw_inf[i+1]
        # Store the result of reading for Pauli stirng
        if read_counter == 3:
            ope_con_list.append(sub_ope_list)
            sub_ope_list = ""
            read_counter = 4
            countdown = 4
        # Ready for reading the coefficient
        if read_counter == 4:
            if countdown == 0:
                read_counter = 1
            else:
                countdown -= 1
    # Convert type of the coefficient reading result into complex
    for i in range(len(convert_list)):
        if convert_list[i][0] != '(':
            num_list.append(complex(convert_list[i]))
        else:
            convert_list[i] = convert_list[i][1:-1]
            num_list.append(complex(convert_list[i]))
    # Remove unnecessary spaces from the Pauli matrix reading result  
    for i in range(len(ope_con_list)):
        ope_con_list[i] = ope_con_list[i][1:]
        if 0 < len(ope_con_list[i]):
            ope_con_list[i] = ope_con_list[i].replace(" ", "")
        ope_list.append(ope_con_list[i])
        
    return num_list, ope_list
```

### **実行例**

In [27]:
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
from pyscf import fci, gto, dft, tddft, scf, tools, mcscf
import pitbe

In [28]:
basis = "sto-3g"
multiplicity = 1
charge = 0
geometry = [["H", [0,0,0]],["H", [0,0,0.865]]]
description  = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_pyscf(molecule,run_scf=1,run_cisd=1,run_ccsd=1,run_fci=1)
fermionic_hamiltonian = get_fermion_operator(molecule.get_molecular_hamiltonian())
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)
bk_hamiltonian = bravyi_kitaev(fermionic_hamiltonian)


In [29]:
print('---Jordan-Wigner Transformation---')
print(jw_hamiltonian)
print('---Reading Result---')
print('Coefficient')
print(pitbe.read_general(str(jw_hamiltonian))[0])
print('Pauli matrix product')
print(pitbe.read_general(str(jw_hamiltonian))[1])

---Jordan-Wigner Transformation---
(-0.2300259962688794+0j) [] +
(-0.04711430043380896+0j) [X0 X1 Y2 Y3] +
(0.04711430043380896+0j) [X0 Y1 Y2 X3] +
(0.04711430043380896+0j) [Y0 X1 X2 Y3] +
(-0.04711430043380896+0j) [Y0 Y1 X2 X3] +
(0.153592312841566+0j) [Z0] +
(0.16276753098254843+0j) [Z0 Z1] +
(0.1135597567900135+0j) [Z0 Z2] +
(0.16067405722382247+0j) [Z0 Z3] +
(0.153592312841566+0j) [Z1] +
(0.16067405722382247+0j) [Z1 Z2] +
(0.1135597567900135+0j) [Z1 Z3] +
(-0.17274355195698893+0j) [Z2] +
(0.16885887724673027+0j) [Z2 Z3] +
(-0.17274355195698887+0j) [Z3]
---Reading Result---
Coefficient
[(-0.2300259962688794+0j), (-0.04711430043380896+0j), (0.04711430043380896+0j), (0.04711430043380896+0j), (-0.04711430043380896+0j), (0.153592312841566+0j), (0.16276753098254843+0j), (0.1135597567900135+0j), (0.16067405722382247+0j), (0.153592312841566+0j), (0.16067405722382247+0j), (0.1135597567900135+0j), (-0.17274355195698893+0j), (0.16885887724673027+0j), (-0.17274355195698887+0j)]
Pauli matrix pr

In [30]:
print('---Bravyi-Kitaev Transformation---')
print(bk_hamiltonian)
print('---Reading Result---')
print('Coefficient')
print(pitbe.read_general(str(bk_hamiltonian))[0])
print('Pauli matrix product')
print(pitbe.read_general(str(bk_hamiltonian))[1])

---Bravyi-Kitaev Transformation---
(-0.2300259962688794+0j) [] +
(0.04711430043380896+0j) [X0 Z1 X2] +
(0.04711430043380896+0j) [X0 Z1 X2 Z3] +
(0.04711430043380896+0j) [Y0 Z1 Y2] +
(0.04711430043380896+0j) [Y0 Z1 Y2 Z3] +
(0.153592312841566+0j) [Z0] +
(0.153592312841566+0j) [Z0 Z1] +
(0.16067405722382247+0j) [Z0 Z1 Z2] +
(0.16067405722382247+0j) [Z0 Z1 Z2 Z3] +
(0.1135597567900135+0j) [Z0 Z2] +
(0.1135597567900135+0j) [Z0 Z2 Z3] +
(0.16276753098254843+0j) [Z1] +
(-0.17274355195698887+0j) [Z1 Z2 Z3] +
(0.16885887724673027+0j) [Z1 Z3] +
(-0.17274355195698893+0j) [Z2]
---Reading Result---
Coefficient
[(-0.2300259962688794+0j), (0.04711430043380896+0j), (0.04711430043380896+0j), (0.04711430043380896+0j), (0.04711430043380896+0j), (0.153592312841566+0j), (0.153592312841566+0j), (0.16067405722382247+0j), (0.16067405722382247+0j), (0.1135597567900135+0j), (0.1135597567900135+0j), (0.16276753098254843+0j), (-0.17274355195698887+0j), (0.16885887724673027+0j), (-0.17274355195698893+0j)]
Pauli m